# Hamiltonian Simulation Via Trotter Decomposition

Given that any Hamiltonian can be decomposed into a sum of Pauli matrices and coefficients:

H = Sum(a*P) where a is a real coefficient and P is a pauli term (any number of pauli matrices tensored together)

The evolution of this Hamiltonian can be simulated via the trotter decomposition:

U(t) = exp(iHt) = (Π exp(iaPt/r))^r

where a is a real coefficient to the Pauli term P, and r is the trotter number

In [222]:
import qiskit
import numpy as np
import sympy
from qutip import *

From: https://arxiv.org/pdf/1001.3855.pdf

We know that the circuit for exp(iZ*Z...Zt) is created simply with CNOT's and an Rz gate

The function below creates this circuit for any arbitrary number of pauli z matrices

In [223]:
def pauli_z_exp(circ, index, t):
    """
    Creates a circuit for exp(i*Z*Z*...*Z*t)
    circ: a quantum circuit
    index: list of indexes in circ that correspond to pauli matrices (I not included)
    t: time (t in exp)
    """
    for i in range(len(index)-1):
        circ.cx(index[i], index[i+1])
    circ.rz(-2*t, index[-1])
    for i in range(len(index)-1, 0, -1):
        qc.cx(index[i-1], index[i])
        
qc = qiskit.QuantumCircuit(3)
pauli_z_exp(qc, [0,1,2], np.pi)
qc.draw()

q_0: ──■─────────────────────────■──
     ┌─┴─┐                     ┌─┴─┐
q_1: ┤ X ├──■───────────────■──┤ X ├
     └───┘┌─┴─┐┌─────────┐┌─┴─┐└───┘
q_2: ─────┤ X ├┤ Rz(-2π) ├┤ X ├─────
          └───┘└─────────┘└───┘

Similarly from: https://arxiv.org/pdf/1001.3855.pdf

We know that any pauli gate can be tranformed into pauli Z gates. X with hadamards, and Y with u2(pi/2) gates.

The function below does this for an arbitrary number of pauli matrices

In [224]:
def pauli_string_exp(paulis, circ, index, t):   
    """
    Creates a circuit for exp(i*P*t) where P is any number of tensored pauli matrices
    circ: a quantum circuit
    index: list of indexes in circ that correspond to pauli matrices
    t: time (t in exp)
    """
    xyz_paulis = []
    xyz_index = []
    for i in range(len(paulis)):
        if paulis[i] != 'I':
            xyz_paulis.append(paulis[i])
            xyz_index.append(index[i])
    
    for i in range(len(xyz_paulis)):
        if xyz_paulis[i] == 'X':
            circ.h(xyz_index[i])
        elif xyz_paulis[i] == 'Y':
            circ.u(np.pi/2, np.pi/2, np.pi/2, xyz_index[i])
            
    pauli_z_exp(circ, xyz_index, t)
    
    for i in range(len(xyz_paulis)):
        if xyz_paulis[i] == 'X':
            circ.h(xyz_index[i])
        elif xyz_paulis[i] == 'Y':
            circ.u(np.pi/2, np.pi/2, np.pi/2, xyz_index[i])
            
qc = qiskit.QuantumCircuit(3)
pauli_string_exp('ZXY', qc, [0,1,2], np.pi)
qc.draw()
    

q_0: ────────────────────■───────────────────────────────■──────────────
           ┌───┐       ┌─┴─┐                           ┌─┴─┐       ┌───┐
q_1: ──────┤ H ├───────┤ X ├──■───────────────■────────┤ X ├───────┤ H ├
     ┌─────┴───┴──────┐└───┘┌─┴─┐┌─────────┐┌─┴─┐┌─────┴───┴──────┐└───┘
q_2: ┤ U(π/2,π/2,π/2) ├─────┤ X ├┤ Rz(-2π) ├┤ X ├┤ U(π/2,π/2,π/2) ├─────
     └────────────────┘     └───┘└─────────┘└───┘└────────────────┘

The function below takes in a hamiltonian decomposed with pauli matrices and creates the full circuit used for simulation

In [225]:
def hamiltonian_sim(hammy, circ, index, t, trotter=1):
    """
    Creates and RETURNS a circuit for a hamiltonian expressed in trotter decomposition
    hammy: dictionary representation of hamiltonian expressed in trotter decomposition
           key is string of pauli matrices P (pauli x is X, pauli y is Y, etc)
           value is weight of P
           e.g. {'XXY': 4, 'IYY': 6, 'XYZ': 7}
    circ: a quantum circuit
    index: list of indexes in circ that correspond to pauli matrices
    t: time (t in exp)
    trotter: the trotter number
    """
    # So I should be able to feed temp into pauli_string_exp then compose it onto circ and not have to return anything.
    # But for some reason, python doesn't like that and it destroys the circuit... I don't know why, I've tried so many
    # different ways of getting this to work, and for some reason this is the only way that it works. So we'll keep it
    temp = qiskit.QuantumCircuit(len(index))
    delta = t/trotter
    for pauli in hammy:
        pauli_string_exp(pauli, circ, range(len(index)), hammy[pauli] * delta)
                
    for i in range(trotter):
        temp.compose(circ, index, inplace=True)
        
    return temp
        
hammy_test = {'XYZ': 2, "YYI": 2}
qc = qiskit.QuantumCircuit(3)
t = hamiltonian_sim(hammy_test, qc, [0,1,2], 1/2*np.pi, 2)
t.draw()

┌───┐                                           ┌───┐       »
q_0: ──────┤ H ├─────────■────────────────────────■────────┤ H ├───────»
     ┌─────┴───┴──────┐┌─┴─┐                    ┌─┴─┐┌─────┴───┴──────┐»
q_1: ┤ U(π/2,π/2,π/2) ├┤ X ├──■──────────────■──┤ X ├┤ U(π/2,π/2,π/2) ├»
     └────────────────┘└───┘┌─┴─┐┌────────┐┌─┴─┐└───┘└────────────────┘»
q_2: ───────────────────────┤ X ├┤ Rz(-π) ├┤ X ├───────────────────────»
                            └───┘└────────┘└───┘                       »
«     ┌────────────────┐                    ┌────────────────┐»
«q_0: ┤ U(π/2,π/2,π/2) ├──■──────────────■──┤ U(π/2,π/2,π/2) ├»
«     ├────────────────┤┌─┴─┐┌────────┐┌─┴─┐├────────────────┤»
«q_1: ┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(-π) ├┤ X ├┤ U(π/2,π/2,π/2) ├»
«     └────────────────┘└───┘└────────┘└───┘└────────────────┘»
«q_2: ────────────────────────────────────────────────────────»
«                                                             »
«           ┌───┐                                           ┌───┐       »
«q_0: ──────┤ H ├─────────■────────────────────────■────────┤ H ├───────»
«     ┌─────┴───┴──────┐┌─┴─┐                    ┌─┴─┐┌─────┴───┴──────┐»
«q_1: ┤ U(π/2,π/2,π/2) ├┤ X ├──■──────────────■──┤ X ├┤ U(π/2,π/2,π/2) ├»
«     └────────────────┘└───┘┌─┴─┐┌────────┐┌─┴─┐└───┘└────────────────┘»
«q_2: ───────────────────────┤ X ├┤ Rz(-π) ├┤ X ├───────────────────────»
«                            └───┘└────────┘└───┘                       »
«     ┌────────────────┐                    ┌────────────────┐
«q_0: ┤ U(π/2,π/2,π/2) ├──■──────────────■──┤ U(π/2,π/2,π/2) ├
«     ├────────────────┤┌─┴─┐┌────────┐┌─┴─┐├────────────────┤
«q_1: ┤ U(π/2,π/2,π/2) ├┤ X ├┤ Rz(-π) ├┤ X ├┤ U(π/2,π/2,π/2) ├
«     └────────────────┘└───┘└────────┘└───┘└────────────────┘
«q_2: ────────────────────────────────────────────────────────
«

Now we will simulate the circuit using qiskit and the circuit method, then with qutip to check our result.

We will start by simulating the hamiltonian: 

H = 2*XZY + 5*ZXX + 2*YXZ

With r = 50 and t = 1/2pi, which should give us an error bound of ε = t^2/r = (1/2pi)^2 / 50 = .0005

In [234]:
qc = qiskit.QuantumCircuit(3)
ham_circ = hamiltonian_sim({"XZY": 2, "ZXX": 5, "YXZ": 2}, qc, [0, 1, 2], 1/(2*np.pi), trotter=50)

ham_circ = ham_circ.reverse_bits()

backend = qiskit.Aer.get_backend('statevector_simulator')
result  = qiskit.execute(ham_circ, backend).result()
vec = result.get_statevector()
vec = vec / vec[0] # global phase
qvec = vec / np.linalg.norm(vec) # normalize
sympy.Matrix(np.round(qvec, 5))

Matrix([
[             0.62069],
[                   0],
[                   0],
[  0.0279 + 0.66576*I],
[                   0],
[-0.26743 + 0.23046*I],
[-0.21432 + 0.01279*I],
[                   0]])

In [214]:
H0 = 2*tensor(sigmax(),sigmaz(),sigmay())
H1 = 5*tensor(sigmaz(),sigmax(),sigmax())
H2 = 2*tensor(sigmay(),sigmax(),sigmaz())

H = H0 + H1 + H2
psi0 = basis(8,0)

t = np.linspace(0, 1/(2*np.pi), 300)

solv = sesolve(H, psi0, t)
sympy.Matrix(np.round(solv.states[-1], 5))

Matrix([
[ 0.6202 - 0.02471*I],
[                  0],
[                  0],
[         -0.66635*I],
[                  0],
[0.27642 + 0.21964*I],
[            0.21465],
[                  0]])

As you can see, our results are very close, though there appears to be an issue with signs.

Let's try again with 2 more hamiltonians:

In [230]:
qc = qiskit.QuantumCircuit(2)
ham_circ = hamiltonian_sim({"ZY": 2, "ZX": 5}, qc, [0, 1], 1/(2*np.pi), trotter=50)

ham_circ = ham_circ.reverse_bits()

backend = qiskit.Aer.get_backend('statevector_simulator')
result  = qiskit.execute(ham_circ, backend).result()
vec = result.get_statevector()
vec = vec / vec[0] # global phase
qvec = vec / np.linalg.norm(vec) # normalize
sympy.Matrix(np.round(qvec, 5))

Matrix([
[             0.65467],
[-0.28551 + 0.69992*I],
[                   0],
[                   0]])

In [231]:
H0 = 2*tensor(sigmaz(),sigmay())
H1 = 5*tensor(sigmaz(),sigmax())

H = H0 + H1
psi0 = basis(4,0)

t = np.linspace(0, 1/(2*np.pi), 300)

solv = sesolve(H, psi0, t)
sympy.Matrix(np.round(solv.states[-1], 5))

Matrix([
[            0.65465],
[0.28075 - 0.70186*I],
[                  0],
[                  0]])

In [232]:
qc = qiskit.QuantumCircuit(4)
ham_circ = hamiltonian_sim({"ZYZZ": 2, "ZXYX": 5}, qc, [0, 1, 2, 3], 1/(2*np.pi), trotter=50)

ham_circ = ham_circ.reverse_bits()

backend = qiskit.Aer.get_backend('statevector_simulator')
result  = qiskit.execute(ham_circ, backend).result()
vec = result.get_statevector()
vec = vec / vec[0] # global phase
qvec = vec / np.linalg.norm(vec) # normalize
sympy.Matrix(np.round(qvec, 5))

Matrix([
[ 0.65465],
[       0],
[       0],
[ 0.00447],
[-0.28072],
[       0],
[       0],
[-0.70186],
[       0],
[       0],
[       0],
[       0],
[       0],
[       0],
[       0],
[       0]])

In [233]:
H0 = 2*tensor(sigmaz(),sigmay(),sigmaz(),sigmaz())
H1 = 5*tensor(sigmaz(),sigmax(),sigmay(),sigmax())

H = H0 + H1
psi0 = basis(16,0)

t = np.linspace(0, 1/(2*np.pi), 300)

solv = sesolve(H, psi0, t)
sympy.Matrix(np.round(solv.states[-1], 5))

Matrix([
[0.65465],
[      0],
[      0],
[      0],
[0.28075],
[      0],
[      0],
[0.70186],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0],
[      0]])

While all of the results are quite close, there does seem to be a continuing issue with signs when simulating the hamiltonians.